In [16]:
import sys

import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table
import pandas as pd

sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")

import GCRCatalogs
#Put nersc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)

import clevar
from clevar import ClCatalog, MemCatalog
from clevar.match import MembershipMatch, get_matched_pairs
from clevar.match import output_matched_catalog
from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf

GCRCatalogs = 1.9.0 | GCR = 0.9.2


In [17]:
cluster_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/cluster_data.fits')
member_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/member_data.fits')
truth_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/truth_data.fits')

In [18]:
match_ID = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/match_ID.fits')

In [19]:
# CLEVAR

min_richness = 5
min_halo_mass = 1e13 #Msun

# Halos/Clusters
#CosmoDC2 + RedMaPPer: 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
RM_cat = ClCatalog.read('/global/homes/c/cinlima/Data/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

#CosmoDC2: 'cosmoDC2_v1.1.4_image'
halo_cat = ClCatalog.read('/global/homes/c/cinlima/Data/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

#Members
RM_cat.read_members('/global/homes/c/cinlima/Data/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
halo_cat.read_members('/global/homes/c/cinlima/Data/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

#Match catalogs
mt = MembershipMatch()

mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/Data/CDC2Data/cosmoDC2_RM_match_cross_membership')
RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

RM_cat
    * ClEvar used in matching: 0.14.1
 * Total objects:    45,081
 * multiple (self):  43,213
 * multiple (other): 43,213
 * unique (self):    43,035
 * unique (other):   43,037
 * cross:            43,035

halo_cat
    * ClEvar used in matching: 0.14.1
 * Total objects:    471,457
 * multiple (self):  75,785
 * multiple (other): 75,785
 * unique (self):    43,037
 * unique (other):   43,035
 * cross:            43,035


In [24]:
match_clevar = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('mass', 'richness', 'redshift'))


In [25]:
selection_ID = match_ID[np.logical_and(np.log(match_ID['richness']) > 5.0,
                                                       np.log(match_ID['richness'] < 5.5))]

selection_ID = selection_ID[np.logical_and(selection_ID['redshift'] > 0.2, selection_ID['redshift'] < 0.4 )]

selection_ID

/tmp/ipykernel_880061/3075592925.py:2: RuntimeWarning: divide by zero encountered in log
  np.log(match_ID['richness'] < 5.5))]


id,cluster_id,halo_id,cluster_ra,ra,cluster_dec,dec,richness,richness_err,halo_mass,is_central,redshift,cluster_redshift,cluster_redshift_err,frac_catalog1
int64,int32,int64,float64,float64,float64,float64,float32,float32,float64,bool,float64,float32,float32,float64
5250182580,14,57800084355,69.34670035077468,69.34670035077468,-33.39340819018279,-33.39340819018279,204.72597,4.495039,364188580734197.2,True,0.3756054176465722,0.3828181,0.00384154,0.9
8562541846,3,101900137401,50.413689701043914,50.413689701043914,-38.520252684731176,-38.520252684731176,266.29504,4.584456,395317641677521.1,True,0.21616988261672043,0.21920915,0.004083831,0.3344155844155844
8812617235,8,536900141373,60.730393791492105,60.730393791492105,-37.49569503313509,-37.49569503313509,253.75546,5.3337297,560738888378050.75,True,0.3329006927699356,0.33059168,0.005507138,0.42342342342342343
9750040774,12,700156401,63.563618571508925,63.563618571508925,-40.402946189791535,-40.402946189791535,203.45981,4.691607,401753499884529.6,True,0.24034691441733247,0.2439135,0.004576755,0.42084942084942084
12000042585,22,242900192401,71.48910979131912,71.48910979131912,-45.107857596384726,-45.107857596384726,170.55482,3.8476717,451815767233262.0,True,0.2261143025679202,0.22532119,0.004205084,0.9257425742574258


In [27]:
selection_clevar = match_ID[np.logical_and(np.log(match_clevar['richness']) > 5.0,
                                                       np.log(match_clevar['richness'] < 5.5))]

selection_clevar = selection_clevar[np.logical_and(selection_clevar['redshift'] > 0.2, selection_clevar['redshift'] < 0.4 )]

# selection_clevar

/tmp/ipykernel_880061/2332066690.py:2: RuntimeWarning: divide by zero encountered in log
  np.log(match_clevar['richness'] < 5.5))]


IndexError: boolean index did not match indexed array along dimension 0; dimension is 38907 but corresponding boolean dimension is 43035